### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Using Ray as a cluster.

In [1]:
import ray

Start a cluster with single worker.

In [ ]:
ray.init(num_cpus=1)

Using Apache Arrow columnar storage. Import pyarrow libraries.

In [1]:
import pyarrow as pa
from pyarrow import csv

Get a Liten Cache Instance

In [2]:
import liten as ten

<frozen importlib._bootstrap>:219: RuntimeWarning: compiletime version 3.6 of module 'liten._liten' does not match runtime version 3.8
<frozen importlib._bootstrap>:219: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [3]:
tc = ten.new_cache()

Added a new cache


In [4]:
tc.show_versions()

'0.0.1'

Read a table into TCache/

In [ ]:
#import functools
def add_table(tcache, table_name, read_func):
    @functools.wraps(read_func)
    def wrap_read_func(*args, **kwargs):
        arrow_table = read_func(*args, **kwargs)
        tcache.add_table(table_name, arrow_table)
        return arrow_table
    return wrap_read_func

#@add_table
#def read_csv(input_file, parse_options):
#    arrow_table = pa.csv.read_csv(input_file=input_file, parse_options=parse_options)
#    return arrow_table

def read_csv(table_name, input_file, parse_options):
    arrow_table = pa.csv.read_csv(input_file=input_file, parse_options=parse_options)
    tc.add_table(table_name, arrow_table)
    return arrow_table

In [ ]:
def liten(tcache, tablename):
    def add_table(read_func):
        @functools.wraps(read_func)
        def wrap_read_func(*args, **kwargs):
            arrow_table = read_func(*args, **kwargs)
            tcache.add_table(table_name, arrow_table)
            return arrow_table
        return wrap_read_func
    return add_table



In [ ]:
def get_tpch_tables():
    table_names = ['lineitem','customer','orders','supplier','nation','region']
    tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/'
    #tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/'
    tpch_tables = []
    for table_name in table_names:
        tpch_tables.append(tpch_dir+table_name+'.tbl')
    return tpch_tables
btable_names = [b'lineitem',b'customer',b'orders',b'supplier',b'nation',b'region']

def read_tpch_tables(tpch_tables):
    csv_options = pa.csv.ParseOptions(delimiter='|')
    arrow_tpch_tables = []
    for i in range(len(tpch_tables)):
        #tc_arrow_tpch_table.append(tc.read_csv(input_file=tpch_table, parse_options=csv_options))
        arrow_table=read_csv(table_name=btable_names[i], input_file=tpch_tables[i], parse_options=csv_options)
        arrow_tpch_tables.append(arrow_table)
    return arrow_tpch_tables

In [13]:
class add_table(object):

    def __init__(self, tcache, table_name):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print ("Inside __init__()")
        self.tcache = tcache
        self.table_name = table_name

    def __call__(self, read_func):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print ("Inside __call__()")
        def wrapped_read_func(*args, **kwargs):
            print ("Inside wrapped_f()")
            for arg in args: 
                print("*argv arg :", arg)
            for key, value in kwargs.items(): 
                print ("%s == %s" %(key, value))
            read_func(*args, **kwargs)
            print ("After f(*args)")
        return wrapped_read_func

@add_table(tc, b'lineitem')
def read_csv(input_file, parse_options):
    pa.csv.read_csv(input_file=input_file, parse_options=parse_options)

    
@add_table(tc, b'lineitem')
def read_csv1(*args, **kwargs):
    pa.csv.read_csv(*args, **kwargs)
    
print ("After decoration")

print ("Preparing to call sayHello()")

print ("after first sayHello() call")
csv_options = pa.csv.ParseOptions(delimiter='|')
read_csv1(input_file='/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/lineitem.tbl', 
         parse_options=csv_options)
print ("after second sayHello() call")

Inside __init__()
Inside __call__()
Inside __init__()
Inside __call__()
After decoration
Preparing to call sayHello()
after first sayHello() call
Inside wrapped_f()
input_file == /mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/lineitem.tbl
parse_options == <pyarrow._csv.ParseOptions object at 0x7fe1d66931b0>
After f(*args)
after second sayHello() call


In [12]:
def add_table(arg1, arg2, arg3):
    def wrap(f):
        print ("Inside wrap()")
        def wrapped_f(*args):
            print ("Inside wrapped_f()")
            print ("Decorator arguments:", arg1, arg2, arg3)
            f(*args)
            print ("After f(*args)")
        return wrapped_f
    return wrap

@decoratorFunctionWithArguments("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print ('sayHello arguments:', a1, a2, a3, a4)
    
sayHello("Hello", "world", "again", "again")

Inside wrap()
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: Hello world again again
After f(*args)


In [ ]:
def get_tpch_tables():
    table_names = ['lineitem','customer','orders','supplier','nation','region']
    tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sfdot001g/'
    #tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/'
    tpch_tables = []
    for table_name in table_names:
        tpch_tables.append(tpch_dir+table_name+'.tbl')
    return tpch_tables
btable_names = [b'lineitem',b'customer',b'orders',b'supplier',b'nation',b'region']

def read_tpch_tables(tpch_tables):
    csv_options = pa.csv.ParseOptions(delimiter='|')
    arrow_tpch_tables = []
    for i in range(len(tpch_tables)):
        #tc_arrow_tpch_table.append(tc.read_csv(input_file=tpch_table, parse_options=csv_options))
        arrow_table=read_csv(table_name=btable_names[i], input_file=tpch_tables[i], parse_options=csv_options)
        arrow_tpch_tables.append(arrow_table)
    return arrow_tpch_tables

Read Arrow table

In [ ]:
%%time
arrow_tpch_tables = read_tpch_tables(get_tpch_tables())

In [ ]:
for i in range(len(arrow_tpch_tables)):
    print(btable_names[i]," Rows=", arrow_tpch_tables[i].num_rows," Cols=", arrow_tpch_tables[i].num_columns)

Liten sample commands

In [ ]:
tc.info()

In [ ]:
%%time
bdim_table_names = [b'customer',b'orders',b'supplier',b'nation',b'region']
for table in bdim_table_names:
    result = tc.make_dtensor(table)
    print(table,"=",result)

In [ ]:
tc.query6()

In [ ]:
tc.query5()

In [ ]:
ray.shutdown()